In [2]:
import tkinter as tk
from tkinter import ttk, messagebox
import re
import sys

# Constants
SCALE = 10**6  # 6 знаков после запятой
MAX_INT = 10**12  # 1e12
MAX_SCALED = MAX_INT * SCALE

NUMBER_RE = re.compile(r'^([+-]?)(\d*)(?:[.,]?(\d*))?$')

# Utility functions

def normalize_input(s: str) -> str:
    if s is None:
        return ''
    s = str(s).strip()
    # remove spaces and non-breaking spaces
    s = s.replace('\u00A0', '').replace(' ', '')
    # replace comma with dot
    s = s.replace(',', '.')
    return s


def parse_scaled_int(s: str):
    """Parse string to scaled integer (value * SCALE).
    Returns (ok: bool, value_or_msg)
    """
    s = normalize_input(s)
    if s == '':
        return False, 'Пустой ввод'
    if 'e' in s.lower():
        return False, 'Экспоненциальная нотация не поддерживается'

    m = NUMBER_RE.match(s)
    if not m:
        return False, 'Неверный формат числа'
    sign = -1 if m.group(1) == '-' else 1
    int_part = m.group(2) or '0'
    frac_part = m.group(3) or ''

    # truncate or pad fractional part to exactly 6 digits (truncate; no rounding)
    if len(frac_part) > 6:
        frac_part = frac_part[:6]
    frac_part = frac_part.ljust(6, '0')

    # strip leading zeros on int part
    int_part = int_part.lstrip('0') or '0'

    try:
        int_val = int(int_part)
        frac_val = int(frac_part)
        scaled = (int_val * SCALE + frac_val) * sign
    except Exception as e:
        return False, 'Число слишком большое или неверно'

    # range check for input number magnitude
    if scaled < -MAX_SCALED or scaled > MAX_SCALED:
        return False, f'Вводимое число выходит за допустимый диапазон ±{MAX_INT}.000000'

    return True, scaled


def format_scaled(scaled: int) -> str:
    sign = '-' if scaled < 0 else ''
    a = abs(scaled)
    int_part = a // SCALE
    frac_part = a % SCALE
    return f"{sign}{int_part}.{str(frac_part).rjust(6, '0')}"


# GUI
class CalculatorApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title('Калькулятор — лабораторная')
        self.resizable(False, False)
        self.create_widgets()

    def create_widgets(self):
        pad = 8
        main = ttk.Frame(self, padding=pad)
        main.grid(row=0, column=0, sticky='nsew')

        # Header with student info (must be inside interface, not window title)
        header = ttk.Frame(main)
        header.grid(row=0, column=0, sticky='w')
        ttk.Label(header, text='ФИО студента:  Крепская Анна Валерьевна', font=('Arial', 10, 'bold')).grid(row=0, column=0, sticky='w')
        ttk.Label(header, text='Курс: 4').grid(row=1, column=0, sticky='w')
        ttk.Label(header, text='Группа: 4').grid(row=2, column=0, sticky='w')
        ttk.Label(header, text='Год: 2025').grid(row=3, column=0, sticky='w')

        # Inputs
        frm = ttk.Frame(main, padding=(0,10,0,0))
        frm.grid(row=1, column=0, sticky='w')

        ttk.Label(frm, text='Первое число (точка или запятая как разделитель)').grid(row=0, column=0, sticky='w')
        self.a_var = tk.StringVar()
        self.a_entry = ttk.Entry(frm, textvariable=self.a_var, width=40)
        self.a_entry.grid(row=1, column=0, sticky='w')

        ttk.Label(frm, text='Второе число (точка или запятая как разделитель)').grid(row=2, column=0, sticky='w', pady=(10,0))
        self.b_var = tk.StringVar()
        self.b_entry = ttk.Entry(frm, textvariable=self.b_var, width=40)
        self.b_entry.grid(row=3, column=0, sticky='w')

        # Operation radio
        opfrm = ttk.Frame(frm)
        opfrm.grid(row=4, column=0, pady=(10,0), sticky='w')
        ttk.Label(opfrm, text='Операция:').grid(row=0, column=0, sticky='w')
        self.op_var = tk.StringVar(value='add')
        ttk.Radiobutton(opfrm, text='Сложение', variable=self.op_var, value='add').grid(row=0, column=1, padx=6)
        ttk.Radiobutton(opfrm, text='Вычитание', variable=self.op_var, value='sub').grid(row=0, column=2, padx=6)

        # Buttons
        btnfrm = ttk.Frame(frm)
        btnfrm.grid(row=5, column=0, pady=(10,0), sticky='w')
        self.compute_btn = ttk.Button(btnfrm, text='Вычислить', command=self.on_compute)
        self.compute_btn.grid(row=0, column=0, padx=(0,6))
        self.copy_btn = ttk.Button(btnfrm, text='Скопировать результат', command=self.copy_result)
        self.copy_btn.grid(row=0, column=1, padx=(0,6))
        self.example_btn = ttk.Button(btnfrm, text='Показать пример (граничный)', command=self.fill_example)
        self.example_btn.grid(row=0, column=2)

        # Message and result
        self.msg_var = tk.StringVar()
        ttk.Label(main, textvariable=self.msg_var, foreground='blue').grid(row=2, column=0, sticky='w', pady=(6,0))

        self.result_var = tk.StringVar()
        ttk.Label(main, text='Результат:').grid(row=3, column=0, sticky='w')
        self.result_label = ttk.Entry(main, textvariable=self.result_var, width=40, state='readonly')
        self.result_label.grid(row=4, column=0, sticky='w')

        # Bindings for Ctrl+V and Ctrl+C on entries to ensure paste normalizes comma->dot and blocks e/E
        for ent in (self.a_entry, self.b_entry):
            ent.bind('<Control-v>', self._paste_normalize)
            ent.bind('<Control-V>', self._paste_normalize)
            ent.bind('<Control-c>', self._copy_selection)
            ent.bind('<Control-C>', self._copy_selection)
            # also handle regular paste via right-click menu? many platforms handle default paste

        # Enter key triggers compute when focus is in entry
        self.a_entry.bind('<Return>', lambda e: self.on_compute())
        self.b_entry.bind('<Return>', lambda e: self.on_compute())

    def _paste_normalize(self, event):
        try:
            clip = self.clipboard_get()
        except tk.TclError:
            return 'break'
        clip = normalize_input(clip)
        # if exponential notation — reject
        if 'e' in clip.lower():
            self.msg_var.set('Экспоненциальная нотация не поддерживается (вставка отклонена)')
            return 'break'
        # insert at current position
        widget = event.widget
        try:
            widget.insert('insert', clip)
        except Exception:
            pass
        return 'break'

    def _copy_selection(self, event):
        w = event.widget
        try:
            sel = w.selection_get()
            self.clipboard_clear()
            self.clipboard_append(sel)
            self.msg_var.set('Скопировано')
        except Exception:
            pass
        return 'break'

    def on_compute(self):
        self.msg_var.set('')
        a_text = self.a_var.get()
        b_text = self.b_var.get()
        ok_a, a_val = parse_scaled_int(a_text)
        if not ok_a:
            self.msg_var.set(f'Ошибка в первом числе: {a_val}')
            return
        ok_b, b_val = parse_scaled_int(b_text)
        if not ok_b:
            self.msg_var.set(f'Ошибка во втором числе: {b_val}')
            return

        # Double-check inputs within allowed range (already done in parse)
        if not (-MAX_SCALED <= a_val <= MAX_SCALED) or not (-MAX_SCALED <= b_val <= MAX_SCALED):
            self.msg_var.set(f'Вводимые числа должны быть в диапазоне ±{MAX_INT}.000000')
            return

        if self.op_var.get() == 'add':
            res = a_val + b_val
        else:
            res = a_val - b_val

        if res < -MAX_SCALED or res > MAX_SCALED:
            self.msg_var.set('Переполнение результата (выходит за допустимый диапазон)')
            self.result_var.set('')
            return

        # Format result without exponential notation, exactly 6 decimal places
        self.result_var.set(format_scaled(res))
        self.msg_var.set('Готово. Результат отображён. Используйте Ctrl+C или кнопку для копирования.')

    def copy_result(self):
        val = self.result_var.get()
        if not val:
            self.msg_var.set('Нет результата для копирования')
            return
        try:
            self.clipboard_clear()
            self.clipboard_append(val)
            self.msg_var.set('Результат скопирован в буфер обмена')
        except Exception:
            self.msg_var.set('Не удалось скопировать результат')

    def fill_example(self):
        self.a_var.set('999999999999.999999')
        self.b_var.set('0.000001')
        self.op_var.set('add')
        self.msg_var.set('Пример заполнен: 999999999999.999999 + 0.000001')


if __name__ == '__main__':
    app = CalculatorApp()
    # position window at center
    try:
        app.update_idletasks()
        w = app.winfo_width()
        h = app.winfo_height()
        ws = app.winfo_screenwidth()
        hs = app.winfo_screenheight()
        x = (ws // 2) - (w // 2)
        y = (hs // 2) - (h // 2)
        app.geometry(f'+{x}+{y}')
    except Exception:
        pass
    app.mainloop()


In [3]:
!pip install pyinstaller

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 4.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pyinstaller --onefile --windowed --name "CalculatorLab" calculator.ipynb


669 INFO: PyInstaller: 6.16.0, contrib hooks: 2025.9
669 INFO: Python: 3.10.0
685 INFO: Platform: Windows-10-10.0.26100-SP0
685 INFO: Python environment: C:\Users\Asus\AppData\Local\Programs\Python\Python310
693 INFO: wrote C:\Users\Asus\CalculatorLab.spec
712 INFO: Module search paths (PYTHONPATH):
['C:\\Users\\Asus\\AppData\\Local\\Programs\\Python\\Python310\\Scripts\\pyinstaller.exe',
 'C:\\Users\\Asus\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'C:\\Users\\Asus\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'C:\\Users\\Asus\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'C:\\Users\\Asus\\AppData\\Local\\Programs\\Python\\Python310',
 'C:\\Users\\Asus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages',
 'C:\\Users\\Asus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32',
 'C:\\Users\\Asus\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\Asus\\AppData\\Local\\Programs\\Pyt

"grep" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
